## <div style="color: #E98E29">**1. Ler PDF completo** </div>

 - <s style="color: #92BB2D">CPFL</s>
 - <s style="color: #92BB2D">COPEL</s>
 - CEMIG
 - <s style="color: #92BB2D">EDP</s>
 - ELEKTRO
 - ENEL

---

## <div style="color: #E98E29">**2. Criar a base**</div>
- <s style="color: #92BB2D">Coletar Meses e ano(s)</s>
- <s style="color: #92BB2D">Coletar Consumos</s>
-  <s style="color: #92BB2D">Coletar Demandas</s>
- <s style="color: #92BB2D">Criar DataFrame</s>
- Adicionar a base de dados
<br>
<br>
---

## <s style="color: #92BB2D">**3. Não adicionar datas repetidas**</s>

<br>

## LER OS PDFS

In [73]:
import sys

classes = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\coleta\scripts\classes"
funcs = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\coleta\scripts\funcoes"
sys.path.append(classes)
sys.path.append(funcs)

In [74]:
import os
import contextlib
import regex as re
import pandas as pd
from funcoes.f_database import *
from typing import Any, List, Tuple
from Fatura import Fatura

join_path = lambda y, x: os.path.join(y, x)
filename = lambda path, filetype: path.split("\\")[-1].split(filetype)[0]

PATH = r"Q:\APOIO ATENDIMENTO\Faturas EcoExp\Setembro"
FOLDERS = os.listdir(PATH)
PATHS = [{filename(x, ".pdf"): join_path(PATH, x)} for x in FOLDERS]

# SCRIPTS

## ADICIONAR IDS

In [ ]:
# ADICIONA NOVOS CLIENTES
df = []
ucs_df = df[["nome"]].drop_duplicates().reset_index(drop=True)
db_path = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\banco_de_dados\distribuidoras.db"
ucs_db = get_clientes_existentes(db_path)
ucs_nova = adiciona_novo_id_clientes(mantem_novos_clientes(ucs_df, ucs_db))
if not (ucs_nova.empty):
    print(f"Clientes a adicionar: {', '.join(ucs_nova.nome.values.tolist())}")
    if salva_novos_clientes(ucs_nova, db_path):
        print("Insercao de Novos Clientes ok")
        ucs_db = get_clientes_existentes(db_path)
    else:
        print("Erro ao salvar valores no banco")
else:
    print("Nenhuma UC nova")

consumos_pdf = criar_id_consumos(df_id=ucs_db, df_consumo=df.drop_duplicates(subset=["nome", "datas"]))
consumos_db = get_consumos_existentes(db_path)
novos_consumos = mantem_novos_consumos(
    novos_consumos=consumos_pdf, consumos_existentes=consumos_db
)
if not(novos_consumos.empty):
    status = salva_novos_consumos(novos_consumos=novos_consumos, db_path=db_path)

## DATAFRAME FINAL

In [75]:
from Identificador import Identificador
from PyPDF2.errors import PdfReadError
DISTRIBUIDORAS = {}
for info in PATHS:
    with contextlib.suppress(IndexError,TypeError, PdfReadError):
        path = info.get(*info.keys())
        DISTRIBUIDORA = Identificador(infos=info).main()(path)
        name = DISTRIBUIDORA.distribuidora
        if name not in DISTRIBUIDORAS.keys():
            DISTRIBUIDORAS[name] = []
        if name != None:
            DISTRIBUIDORAS[name].append(DISTRIBUIDORA)

# NOVAS CLASSES - DESENVOLVIMENTO

In [ ]:
import regex as re
from Fatura import Fatura
from typing import Any, List, Tuple
from pandas import DataFrame, Series


class CEMIG(Fatura):
    def __init__(self, path: str):
        super().__init__(path)
        self.split_str = lambda x: re.split("\s+", x)[:-1]
    
    @Fatura.distribuidora.getter
    def distribuidora(self) -> str:
        return self.__class__.__name__

    @Fatura.consumo.setter
    def consumo(self, consumo: DataFrame):
        self._consumo = consumo
    
    @Fatura.data.setter
    def data(self, datas: Series):
        self._data = datas

    @Fatura.demanda.setter
    def demanda(self, demanda: DataFrame):
        self._demanda = demanda

    @Fatura.medida_consumo.setter
    def medida_consumo(self, indice: int):
        self._medida_consumo = re.search("energia\((\w*)\)", self._primeira_pagina).group(1)
    
    @Fatura.medida_demanda.setter
    def medida_demanda(self, indice: int):
        self._medida_demanda = re.search("demanda\((\w*)\)", self._primeira_pagina).group(1)

    @Fatura.nome.setter
    def nome(self, flag: Any):
        self._nome = re.search("cliente: (\w.*) unidade:", self._ultima_pagina).group(1)

    @Fatura.ths.setter
    def ths(self, texto_a_encontrar: str):
        self._ths = re.search(texto_a_encontrar, self._primeira_pagina).group(1)
           
    def main(self):
        idx_0, idx_1 = self.indices()
        dados_lista = list(map(self.split_str, self.dados_lista(idx_0, idx_1)))
        cols = ["datas", "demanda_ponta", "demanda_fora_de_ponta", "consumo_ponta", "consumo_fora_de_ponta"]
        df = DataFrame(dados_lista, columns=cols)
        df.datas = df.datas.apply(self.transforma_data)
        
        self.consumo = df[["datas", "consumo_ponta", "consumo_fora_de_ponta"]]
        self.demanda = df[["datas", "demanda_ponta", "demanda_fora_de_ponta"]]
        self.data = df.datas
        self.ths = "ths (\w+)\s"
        
        self.nome = None
        self.medida_demanda = None
        self.medida_consumo = None

    def dados_lista(self, idx_0: int, idx_1: int) -> List[str]:
        return self._primeira_pagina[idx_0: idx_1].split("\n")

    def indices(self) -> Tuple[int, int]:
        encontrar = "hp hfp hp hfp hr"
        idx_0 = self._primeira_pagina.find(encontrar) + len(encontrar) + 1
        idx_1 = self._primeira_pagina.find("reservado ao fisco")
        return idx_0, idx_1
    

In [ ]:
from EDP import EDP
from CPFL import CPFL
from COPEL import COPEL
from CEMIG import CEMIG


# for cemig in DISTRIBUIDORAS['CEMIG']:
#     caminho = cemig._caminho
#     cmg = CEMIG(caminho)
#     cmg.main()

for cpfl in DISTRIBUIDORAS['CPFL']:
    caminho = cpfl._caminho
    cpfl = CPFL(caminho)
    cpfl.main()

# for edp in DISTRIBUIDORAS['EDP']:
#     caminho = edp._caminho
#     edp = EDP(caminho)
#     edp.main()

# for copel in DISTRIBUIDORAS['COPEL']:
#     caminho = copel._caminho
#     copel = COPEL(caminho)
#     copel.main()

In [ ]:
idx = cpfl._texto_posicao("consumo fora de ponta")

string = cpfl.ultima_pagina[idx:][:re.search("\n\w.*dias", cpfl.ultima_pagina[idx:]).span()[0]]
datas_lista = cpfl.datas_texto(re.sub("ll.*", "", string).split("\n")[1:])
datas_series = pd.Series(datas_lista, name="data")
cpfl._data = datas_series.apply(cpfl.transforma_data)

In [ ]:
cpfl.data

In [ ]:
erro = cpfl._caminho 